In [1]:
# PCB failure detection based on yolo12

#!git clone https://github.com/ultralytics/yolov12  # clone
%cd yolov12
!pip install -r requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.5.1+cu121 (NVIDIA GeForce RTX 3070 Ti Laptop GPU)


#Project: PCB Failure Detection and Analysis at EOL based on YOLOv12



###The objective of this project is to find out the failure reason of the PCB and optimize the accuracy of the model by fine-tuning with different pre-train models. The dataset used is from PKU-Market


###Input：images dataset of the failure PCB；Output：the reason of failure（short, open, missing hole, mouse bite, spurious copper)
###dataset: https://robotics.pkusz.edu.cn/resources/dataset/

###Process:
####1. Preprocess the image data
####2. Split the dataset into train, validation and test. Using pretrain model to train the model, then optimize it using best hyperparameters. Test on the test data.
####3. Evaluation matrix includes precision, recall, F1 score


In [3]:
# 🚀 YOLOv12 PCB缺陷检测系统 (带评估报告版)

import os
import cv2
import torch
import random
import numpy as np
from ultralytics import YOLO
from PIL import Image
import shutil
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from fpdf import FPDF
import json
from collections import defaultdict

class Config:
    # --- 路径设置 ---
    BASE_DIR = "PKU-Market-PCB_coco"
    DATA_DIR = os.path.join(BASE_DIR, "images")
    ANNOT_DIR = os.path.join(BASE_DIR, "pcb_cocoanno")
    TRAIN_JSON = os.path.join(ANNOT_DIR, "train.json")
    VAL_JSON = os.path.join(ANNOT_DIR, "val.json")

    # --- 模型设置 ---
    MODEL_PATH = "yolov12n.pt"  # 您可以尝试 yolov12s.pt 或 yolov12m.pt 以获得更高精度
    IMAGE_SIZE = 640
    BATCH_SIZE = 8   #
    EPOCHS = 100     # 建议增加 Epochs

    # --- 数据集划分设置 ---
    # 数据已经预先分好了 train/val
    # 将从 val.json 中再分出一个 test 集
    TEST_SPLIT_RATIO = 0.5 # 0.5 = 50% 的 val.json 数据将用于测试

def verify_model():
    if not os.path.exists(Config.MODEL_PATH):
        raise FileNotFoundError(f"模型文件 {Config.MODEL_PATH} 不存在")
    try:
        model = YOLO(Config.MODEL_PATH)
        # 移除此行: model.model.nc = len(Config.CLASS_NAMES)
        return model 
    except Exception as e:
        raise RuntimeError(f"模型加载失败: {e}")

def prepare_dataset():
    """
    (COCO JSON 版本)
    解析 train.json 和 val.json, 转换标注为 YOLO 格式,
    并创建 train/val/test 
    """
    
    print("🚀 开始处理 COCO JSON 数据集...")
    
    # 1. 创建所有必需的目录
    for split in ['train', 'val', 'test']:
        os.makedirs(f'dataset/images/{split}', exist_ok=True)
        os.makedirs(f'dataset/labels/{split}', exist_ok=True)

    # 2. 加载 JSON 文件
    try:
        with open(Config.TRAIN_JSON, 'r') as f:
            train_data = json.load(f)
        with open(Config.VAL_JSON, 'r') as f:
            val_data = json.load(f)
    except FileNotFoundError as e:
        print(f"❌ 错误: 无法找到 JSON 文件: {e}")
        return None # <--- 如果失败，返回 None

    # 3. 从 train.json 提取类别
    if 'categories' not in train_data:
        print(f"❌ 错误: {Config.TRAIN_JSON} 中缺少 'categories' 键。")
        return None # <--- 如果失败，返回 None
        
    categories = sorted(train_data['categories'], key=lambda x: x['id'])
    
    # 类别 ID 映射 (coco_id -> yolo_id)
    class_id_map = {cat['id']: i for i, cat in enumerate(categories)}
    class_names = [cat['name'] for cat in categories]
    
    if not class_names:
        print(f"❌ 错误: 未能在 {Config.TRAIN_JSON} 中找到任何类别。")
        return None # <--- 如果失败，返回 None
        
    print(f"✅ 成功加载类别: {class_names}")

    # 4. 创建 YAML 配置文件
    yaml_content = f"""
path: {os.path.abspath('dataset')}
train: images/train
val: images/val
test: images/test

nc: {len(class_names)}
names: {class_names}
"""
    with open("dataset/pcb_defect.yaml", "w") as f:
        f.write(yaml_content)
    print("✅ dataset/pcb_defect.yaml 文件已创建。")

    # 5. 定义一个辅助函数来处理数据
    def process_coco_data(data, split):
        """
        处理COCO数据 (images 和 annotations), 
        将其保存到 YOLO 格式的目录中。
        """
        print(f"--- 正在处理 {split} 数据... ---")
        
        image_map = {img['id']: img for img in data['images']}
        annotation_map = defaultdict(list)
        for ann in data['annotations']:
            annotation_map[ann['image_id']].append(ann)
            
        file_count = 0
        
        for image_id, img_info in image_map.items():
            file_name = img_info['file_name']
            img_width = img_info['width']
            img_height = img_info['height']
            
            src_img_path = os.path.join(Config.DATA_DIR, file_name)
            if not os.path.exists(src_img_path):
                print(f"⚠️ 警告: 图像 {src_img_path} 在 JSON 中被引用, 但在 images 目录中未找到。")
                continue

            annotations = annotation_map.get(image_id, [])
            
            yolo_labels = []
            for ann in annotations:
                coco_cat_id = ann['category_id']
                yolo_cat_id = class_id_map.get(coco_cat_id)
                
                if yolo_cat_id is None:
                    print(f"⚠️ 警告: 在 {file_name} 中找到未知 category_id: {coco_cat_id}")
                    continue
                
                x_min, y_min, w, h = ann['bbox']
                
                x_center_norm = (x_min + w / 2) / img_width
                y_center_norm = (y_min + h / 2) / img_height
                w_norm = w / img_width
                h_norm = h / img_height
                
                x_center_norm = max(0, min(1, x_center_norm))
                y_center_norm = max(0, min(1, y_center_norm))
                w_norm = max(0, min(1, w_norm))
                h_norm = max(0, min(1, h_norm))

                yolo_labels.append(
                    f"{yolo_cat_id} {x_center_norm:.6f} {y_center_norm:.6f} {w_norm:.6f} {h_norm:.6f}"
                )

            base_name = os.path.splitext(file_name)[0]
            dest_label_path = os.path.join("dataset/labels", split, base_name + ".txt")
            with open(dest_label_path, 'w') as f:
                f.write("\n".join(yolo_labels))
                
            dest_img_path = os.path.join("dataset/images", split, file_name)
            shutil.copy(src_img_path, dest_img_path)
            
            file_count += 1
            
        return file_count

    # 6. 处理训练集 (train.json)
    train_count = process_coco_data(train_data, 'train')
    
    # 7. 拆分验证集 (val.json) 为新的 val 和 test
    val_images = val_data['images']
    random.shuffle(val_images) 
    
    split_idx = int(len(val_images) * (1 - Config.TEST_SPLIT_RATIO))
    new_val_images = val_images[:split_idx]
    new_test_images = val_images[split_idx:]
    
    new_val_image_ids = {img['id'] for img in new_val_images}
    new_test_image_ids = {img['id'] for img in new_test_images}
    
    new_val_annotations = [ann for ann in val_data['annotations'] if ann['image_id'] in new_val_image_ids]
    new_test_annotations = [ann for ann in val_data['annotations'] if ann['image_id'] in new_test_image_ids]

    new_val_data = {'images': new_val_images, 'annotations': new_val_annotations, 'categories': categories}
    new_test_data = {'images': new_test_images, 'annotations': new_test_annotations, 'categories': categories}
    
    # 8. 处理新的 val 和 test 集
    val_count = process_coco_data(new_val_data, 'val')
    test_count = process_coco_data(new_test_data, 'test')
    
    print("\n" + "="*30)
    print("✅ 数据集处理完成!")
    print(f"  训练集: {train_count} 张图像")
    print(f"  验证集: {val_count} 张图像")
    print(f"  测试集: {test_count} 张图像")
    print("="*30 + "\n")
    
    return class_names

def evaluate_model(model, data_loader, split_name):
    """评估模型并生成指标"""
    results = model.val(data=data_loader)
    
    # 获取评估指标
    metrics = {
        'precision': results.results_dict['metrics/precision(B)'],
        'recall': results.results_dict['metrics/recall(B)'],
        'mAP50': results.results_dict['metrics/mAP50(B)'],
        'mAP50-95': results.results_dict['metrics/mAP50-95(B)'],
        'confusion_matrix': results.confusion_matrix
    }
    
    # --- 这是修改的部分 ---
    # 从模型本身获取类别名称, 而不是 Config
    class_names = model.names.values()
    
    # 保存混淆矩阵图像
    plt.figure(figsize=(10, 8))
    sns.heatmap(metrics['confusion_matrix'], annot=True, fmt='g', 
                xticklabels=class_names,  # <--- 修改
                yticklabels=class_names)  # <--- 修改
    plt.title(f'Confusion Matrix ({split_name})')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    cm_path = f"results/confusion_matrix_{split_name}.png"
    plt.savefig(cm_path)
    plt.close()
    
    return metrics

def generate_report(metrics_dict):
    """生成PDF评估报告"""
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, "PCB缺陷检测评估报告", 0, 1, 'C')
    pdf.ln(10)
    
    # 添加指标表格
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(0, 10, "模型性能指标", 0, 1)
    pdf.set_font("Arial", '', 10)
    
    col_widths = [40, 30, 30, 30]
    headers = ["指标", "验证集", "测试集", "差值"]
    
    # 表头
    for i, header in enumerate(headers):
        pdf.cell(col_widths[i], 10, header, 1, 0, 'C')
    pdf.ln()
    
    # 表格内容
    for metric in ['precision', 'recall', 'mAP50', 'mAP50-95']:
        val = metrics_dict['val'].get(metric, 0)
        test = metrics_dict['test'].get(metric, 0)
        diff = test - val
        
        pdf.cell(col_widths[0], 10, metric.capitalize(), 1)
        pdf.cell(col_widths[1], 10, f"{val:.4f}", 1, 0, 'C')
        pdf.cell(col_widths[2], 10, f"{test:.4f}", 1, 0, 'C')
        pdf.cell(col_widths[3], 10, f"{diff:+.4f}", 1, 0, 'C')
        pdf.ln()
    
    # 添加混淆矩阵
    pdf.add_page()
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(0, 10, "验证集混淆矩阵", 0, 1)
    pdf.image(f"results/confusion_matrix_val.png", x=10, w=180)
    
    pdf.add_page()
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(0, 10, "测试集混淆矩阵", 0, 1)
    pdf.image(f"results/confusion_matrix_test.png", x=10, w=180)
    
    report_path = "results/evaluation_report.pdf"
    pdf.output(report_path)
    print(f"✅ 评估报告已生成: {report_path}")

def main():
    # 初始化
    os.makedirs("results", exist_ok=True)
    
    # 1. 验证模型 (此时不设置 nc)
    try:
        model = verify_model()
    except Exception as e:
        print(f"❌ 模型初始化失败: {e}")
        return

    # 2. 准备数据集, 并获取类别名称
    try:
        class_names = prepare_dataset() # <--- 修改
        if class_names is None: # <--- 检查 prepare_dataset 是否成功
            print("❌ 数据集准备失败, 停止运行。")
            return
    except Exception as e:
        print(f"❌ 数据集准备失败: {e}")
        return

    # 3. --- 这是新增的部分 ---
    # 在训练前, 使用从数据集中读取到的类别数量来设置模型
    try:
        model.model.nc = len(class_names) 
        print(f"✅ 模型类别数已设置为: {model.model.nc}")
    except Exception as e:
        print(f"❌ 设置模型类别数失败: {e}")
        return

    # 4. 训练模型
    print("🚀 开始训练...")
    try:
        model.train(
            data="dataset/pcb_defect.yaml",
            epochs=Config.EPOCHS,
            imgsz=Config.IMAGE_SIZE,
            batch=Config.BATCH_SIZE,
            device="cuda" if torch.cuda.is_available() else "cpu",
            workers=0
        )
    except Exception as e:
        print(f"❌ 训练失败: {e}")
        return

if __name__ == "__main__":
    main()



if __name__ == "__main__":
    main()

🚀 开始处理 COCO JSON 数据集...
✅ 成功加载类别: ['missing_hole', 'mouse_bite', 'open_circuit', 'short', 'spur', 'spurious_copper']
✅ dataset/pcb_defect.yaml 文件已创建。
--- 正在处理 train 数据... ---
--- 正在处理 val 数据... ---
--- 正在处理 test 数据... ---

✅ 数据集处理完成!
  训练集: 555 张图像
  验证集: 69 张图像
  测试集: 69 张图像

✅ 模型类别数已设置为: 6
🚀 开始训练...
Ultralytics 8.3.218  Python-3.12.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/pcb_defect.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.

In [7]:
import shutil
import os

# 创建保存目录
os.makedirs("saved_models", exist_ok=True)
source_file_path = r"G:\runs\detect\train8\weights\best.pt"
# 复制最佳模型到指定目录
shutil.copy(
    source_file_path,  # <-- 使用这个新的、完整的路径
    "saved_models/pcb_defect_detection_best.pt"
)
print("✅ 模型已保存到 saved_models/")

✅ 模型已保存到 saved_models/


In [8]:
from ultralytics import YOLO

# 1. 加载您训练好的最佳模型
# (请确保 'best.pt' 路径正确)
model_path = 'saved_models/pcb_defect_detection_best.pt' 
model = YOLO(model_path)

# 2. 在 'test' 数据集上运行标准评估
# (这会计算 mAP, Precision, Recall)
print("🚀 正在 'test' 数据集上运行标准 mAP 评估...")
metrics = model.val(
    split='test', 
    data='dataset/pcb_defect.yaml'
)

# 3. 打印真实的评估结果
print("\n--- 真实的评估指标 (mAP) ---")
print(f"  mAP50-95: {metrics.box.map:.4f}")  
print(f"  mAP50:    {metrics.box.map50:.4f}") 
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall:    {metrics.box.mr:.4f}")
print("---------------------------------")

🚀 正在 'test' 数据集上运行标准 mAP 评估...
Ultralytics 8.3.218  Python-3.12.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
YOLOv12n summary (fused): 159 layers, 2,557,898 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 109.829.9 MB/s, size: 1344.5 KB)
val: Scanning G:\Study\AI for Health\project\PCBFailure\dataset\labels\test... 137 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 137/137 425.8it/s 0.3s0.0s
val: New cache created: G:\Study\AI for Health\project\PCBFailure\dataset\labels\test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 9/9 1.8it/s 4.9s0.2s
                   all        137        587      0.911      0.875      0.924      0.487
          missing_hole         23         97      0.971       0.99      0.986      0.605
            mouse_bite         23        106      0.926      0.802      0.884      0.468
          open_circuit         22         89 

In [9]:
import torch
from ultralytics import YOLO
import os
import numpy as np
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from fpdf import FPDF
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import datetime

# 使用基本PDF类（完全避免中文和特殊字符）
class SimplePDF(FPDF):
    def __init__(self):
        super().__init__()
        self.set_font('Arial', '', 12)  # 仅使用基本字体

class TestConfig:
    MODEL_PATH = "saved_models/pcb_defect_detection_best.pt"
    DATA_YAML = "dataset/pcb_defect.yaml"
    TEST_SET_PATH = "dataset/images/test"
    RESULTS_DIR = "test_results"
    CONF_THRESH = 0.25
    IOU_THRESH = 0.5

def test_model():
    os.makedirs(TestConfig.RESULTS_DIR, exist_ok=True)
    
    print("Loading model:", TestConfig.MODEL_PATH)
    model = YOLO(TestConfig.MODEL_PATH)
    
    test_images = [os.path.join(TestConfig.TEST_SET_PATH, f) 
                  for f in os.listdir(TestConfig.TEST_SET_PATH) 
                  if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    print(f"Found {len(test_images)} test images")
    
    all_true_labels = []
    all_pred_labels = []
    unique_labels = set()
    
    for img_path in tqdm(test_images, desc="Processing"):
        img_name = os.path.basename(img_path)
        true_label = None
        
        for class_name in model.names.values():
            if class_name.lower() in img_name.lower():
                true_label = list(model.names.values()).index(class_name)
                unique_labels.add(true_label)
                break
        
        if true_label is None:
            continue
        
        results = model.predict(
            source=img_path,
            conf=TestConfig.CONF_THRESH,
            iou=TestConfig.IOU_THRESH,
            verbose=False
        )
        
        pred_label = None
        for result in results:
            if result.boxes is not None and len(result.boxes) > 0:
                max_conf_idx = np.argmax(result.boxes.conf.cpu().numpy())
                pred_label = int(result.boxes.cls[max_conf_idx].item())
                unique_labels.add(pred_label)
        
        all_true_labels.append(true_label)
        all_pred_labels.append(pred_label if pred_label is not None else true_label)
    
    class_names = [model.names[i] for i in sorted(unique_labels)]
    print(f"Detected classes: {class_names}")
    
    print("\nCalculating metrics...")
    accuracy = np.mean(np.array(all_true_labels) == np.array(all_pred_labels))
    precision = precision_score(all_true_labels, all_pred_labels, average='weighted', labels=sorted(unique_labels))
    recall = recall_score(all_true_labels, all_pred_labels, average='weighted', labels=sorted(unique_labels))
    f1 = f1_score(all_true_labels, all_pred_labels, average='weighted', labels=sorted(unique_labels))
    
    class_report = classification_report(
        all_true_labels, 
        all_pred_labels, 
        target_names=class_names,
        output_dict=True,
        labels=sorted(unique_labels)
    )
    
    # Create confusion matrix
    confusion_matrix = np.zeros((len(unique_labels), len(unique_labels)), dtype=int)
    for true, pred in zip(all_true_labels, all_pred_labels):
        if pred is not None:
            true_idx = list(sorted(unique_labels)).index(true)
            pred_idx = list(sorted(unique_labels)).index(pred)
            confusion_matrix[true_idx][pred_idx] += 1
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    confusion_matrix_path = os.path.join(TestConfig.RESULTS_DIR, 'confusion_matrix.png')
    plt.savefig(confusion_matrix_path, bbox_inches='tight', dpi=300)
    plt.close()
    
    print("Generating report...")
    generate_pdf_report({
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'class_report': class_report,
        'confusion_matrix': confusion_matrix_path,
        'class_names': class_names,
        'test_images_count': len(test_images)
    })
    
    print(f"Test completed! Results saved in: {TestConfig.RESULTS_DIR}")

def generate_pdf_report(metrics):
    pdf = SimplePDF()
    pdf.add_page()
    
    # Add metadata
    pdf.cell(0, 10, f"Model path: {TestConfig.MODEL_PATH}", 0, 1)
    pdf.cell(0, 10, f"Test set size: {metrics['test_images_count']}", 0, 1)
    pdf.cell(0, 10, f"Test time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", 0, 1)
    pdf.ln(15)
    
    # Add overall metrics
    pdf.set_font('', 'B', 14)
    pdf.cell(0, 10, "Overall Metrics", 0, 1)
    pdf.set_font('', '', 12)
    
    col_width = 45
    row_height = 10
    pdf.cell(col_width, row_height, "Accuracy", 1)
    pdf.cell(col_width, row_height, f"{metrics['accuracy']:.4f}", 1, 1)
    pdf.cell(col_width, row_height, "Precision", 1)
    pdf.cell(col_width, row_height, f"{metrics['precision']:.4f}", 1, 1)
    pdf.cell(col_width, row_height, "Recall", 1)
    pdf.cell(col_width, row_height, f"{metrics['recall']:.4f}", 1, 1)
    pdf.cell(col_width, row_height, "F1 Score", 1)
    pdf.cell(col_width, row_height, f"{metrics['f1']:.4f}", 1, 1)
    pdf.ln(15)
    
    # Add classification report
    pdf.set_font('', 'B', 14)
    pdf.cell(0, 10, "Classification Report", 0, 1)
    
    # Table header
    pdf.set_font('', 'B', 10)
    pdf.cell(35, 10, "Class", 1)
    pdf.cell(35, 10, "Precision", 1)
    pdf.cell(35, 10, "Recall", 1)
    pdf.cell(35, 10, "F1-Score", 1)
    pdf.cell(35, 10, "Support", 1)
    pdf.ln()
    
    # Table content
    pdf.set_font('', '', 10)
    for class_name in metrics['class_names']:
        class_data = metrics['class_report'][class_name]
        pdf.cell(35, 10, class_name, 1)
        pdf.cell(35, 10, f"{class_data['precision']:.4f}", 1)
        pdf.cell(35, 10, f"{class_data['recall']:.4f}", 1)
        pdf.cell(35, 10, f"{class_data['f1-score']:.4f}", 1)
        pdf.cell(35, 10, f"{int(class_data['support'])}", 1)
        pdf.ln()
    
    # Add averages
    pdf.set_font('', 'B', 10)
    for avg_type in ['macro avg', 'weighted avg']:
        avg_data = metrics['class_report'][avg_type]
        pdf.cell(35, 10, avg_type, 1)
        pdf.cell(35, 10, f"{avg_data['precision']:.4f}", 1)
        pdf.cell(35, 10, f"{avg_data['recall']:.4f}", 1)
        pdf.cell(35, 10, f"{avg_data['f1-score']:.4f}", 1)
        pdf.cell(35, 10, f"{int(avg_data['support'])}", 1)
        pdf.ln()
    
    # Add confusion matrix
    pdf.add_page()
    pdf.set_font('', 'B', 14)
    pdf.cell(0, 10, "Confusion Matrix", 0, 1)
    pdf.image(metrics['confusion_matrix'], x=10, y=30, w=180)
    
    # Save report
    report_path = os.path.join(TestConfig.RESULTS_DIR, "evaluation_report.pdf")
    pdf.output(report_path)
    return report_path

if __name__ == "__main__":
    test_model()

Loading model: saved_models/pcb_defect_detection_best.pt
Found 137 test images


Processing: 100%|██████████| 137/137 [00:06<00:00, 20.18it/s]
g:\Study\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
g:\Study\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
g:\Study\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
g:\Study\Python\Lib\site-pack

Detected classes: ['missing_hole', 'mouse_bite', 'open_circuit', 'short', 'spur', 'spurious_copper']

Calculating metrics...
Generating report...
Test completed! Results saved in: test_results


## 📥 Upload Zipped PCB Images

Please zip your PCB image files on your local machine and upload the zip file using the code cell below. The zip file will be extracted into the `/content/data` directory.

In [ ]:
from google.colab import files
import os
import zipfile

# Upload the zip file
print("Please upload the zip file containing your PCB images...")
uploaded = files.upload()

# Assume only one file is uploaded
zip_filename = list(uploaded.keys())[0]

# Define the extraction directory
extract_dir = '/content/data'
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
print(f"Extracting {zip_filename} to {extract_dir}...")
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"✅ Successfully extracted {zip_filename} to {extract_dir}")

# Remove the uploaded zip file
os.remove(zip_filename)